In [3]:
import tweepy
import pandas
import csv
import re #regular expression
import nltk
from nltk.corpus import stopwords
##nltk.download('stopwords')# Download stopwords
from textblob import TextBlob
import sys

In [4]:
consumer_key = 'xxx'
consumer_secret = 'xxx'
access_token = 'xxx'
access_secret = 'xxx'
access_token_secret=access_secret

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, wait_on_rate_limit=True,wait_on_rate_limit_notify=True )

In [5]:
non_bmp_map = dict.fromkeys(range(0x10000, sys.maxunicode + 1), 0xfffd)
##print(x.translate(non_bmp_map))

In [6]:
def cleaning(tweet):
    ##Handle Emoticons and Emojis
    #HappyEmoticons
    emoticons_happy = set([
        ':-)', ':)', ';)', ':o)', ':]', ':3', ':c)', ':>', '=]', '8)', '=)', ':}',
        ':^)', ':-D', ':D', '8-D', '8D', 'x-D', 'xD', 'X-D', 'XD', '=-D', '=D',
        '=-3', '=3', ':-))', ":'-)", ":')", ':*', ':^*', '>:P', ':-P', ':P', 'X-P',
        'x-p', 'xp', 'XP', ':-p', ':p', '=p', ':-b', ':b', '>:)', '>;)', '>:-)',
        '<3'
        ])
    
    #Sad Emoticons
    emoticons_sad = set([
        ':L', ':-/', '>:/', ':S', '>:[', ':@', ':-(', ':[', ':-||', '=L', ':<',
        ':-[', ':-<', '=\\', '=/', '>:(', ':(', '>.<', ":'-(", ":'(", ':\\', ':-c',
        ':c', ':{', '>:\\', ';('
        ])
    
    ##Emoji patterns
    #combine sad and happy emoticons
    emoticons = emoticons_happy.union(emoticons_sad)
    
    emoji_pattern = re.compile("["
             u"\U0001F600-\U0001F64F"  # emoticons
             u"\U0001F300-\U0001F5FF"  # symbols & pictographs
             u"\U0001F680-\U0001F6FF"  # transport & map symbols
             u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
             u"\U00002702-\U000027B0"
             u"\U000024C2-\U0001F251"                  
             "]+", flags=re.UNICODE)

##    extract_link
##    regex = r' https?://[^\s<>"]+|www\.[^\s<>"]+ '
##    match = re.search(regex, tweet.full_text)
##    if match:
##        print("links: ", match)

    #replace consecutive non-ASCII characters with a space
    tweet1 = re.sub(r'[^\x00-\x7F]+|,Ä¶',' ', tweet)
##    print("a1             ",tweet1)

    #remove emojis from tweet
    tweet2 = emoji_pattern.sub(r'', tweet1)   
##    print("a2             ",tweet2)

    #utility function to clean tweet text by removing links, special characters using simple regex statements. 
    tweet3=re.sub("(@[A-Za-z0-9_]+)|([^0-9A-Za-z_ \t]) |(\w+:\/\/\S+)", " ", tweet2) 
##    print("a3             ",tweet3)

    #removing numbers
    tweet4= re.sub(" (\d+) | (\d+)(.)(\d+) ", ' ',tweet3) # number alone
    tweet5= re.sub("(\d+)", ' ',tweet4) # number attached
##    print("a4             ",tweet5)

    # removing some charachters 
####    tweet6= re.sub(".", ' ',tweet4) # !!!! remove all
    tweet6= re.sub("[\";:\|,~!=\-+#$%&*\(\)?\.\/]+", ' ',tweet5)
##    print("a5             ",tweet6)
   
    # Create a sublist of lower case words for each tweet
    words_in_tweet = tweet6.lower().split()
##    print("a5    ",words_in_tweet)

    stop_words = set(stopwords.words('english'))
    tweet_cleaned=''
    for word in words_in_tweet:
        if word not in stop_words:
            tweet_cleaned +=' '+ word       
##    print("a6    ",tweets_cleaned)

    return tweet_cleaned

In [ ]:
# we can feed the keywords and other info necesaary for runing the code manually or by the input command or by the reading from info file
##KEYWORDS_str = input("Insert your keywords:") 
##LANG = input("Insert the language code:")
##LOC=input("Insert locations(s):")

Info = pandas.read_csv('Info.csv', sep=';', na_values=".")
Info.shape
Info.columns
##print(Info['keywords'])
print(Info['keywords'][1])
KEYWORDS_str=Info['keywords'][1]
KEYWORDS_list=KEYWORDS_str.split(',')
print(Info['language'][1])
LANG=Info['language'][1]
print(Info['location'][1])
LOC=Info['location'][1]
print()

In [ ]:
search_term = "*  -filter:retweets"
##search_term = "'#entrepreneur' OR business owner'' OR 'founded' OR 'founder'  -filter:retweets"
tweets = tweepy.Cursor(api.search,
                   q=search_term,
                   result_type="recent",
                   count=100,
                   lang=LANG,
                   tweet_mode="extended",
                   ).items() 

In [ ]:
#for the first time for creating the csv file, uncemment this block, then comment this block so that even after interruption due to interenet disconnection, twitter API termination and ... tweets can added to the file.

##csvFile_Loc = open(LOC+'.csv', 'w')
##csvWriter = csv.writer(csvFile_Loc, delimiter=';', quoting=csv.QUOTE_MINIMAL)
##csvWriter.writerow(["user screen_name",
##                    "user name",
##                    "user id",
##                    "tweet",
##                    "cleaned tweet",
##                    "language",
##                    "created at",
##                    "full name of the city for tweet",
##                    "country code for tweet",
##                    "location in profile",
##                    "profile description"])

In [ ]:
# Collecting target tweets
n_total=0
n_target=0
for tweet in tweets:
    n_total +=1
##    print(tweet.full_text.translate(non_bmp_map))
##    print(tweet.user.screen_name.translate(non_bmp_map))
##    print()
    if LOC in tweet.user.location:
####        print(tweet)
####        print(tweet.author)
####        print(tweet.geo)
####        print(tweet.place)
####        print(tweet.metadata)       

        try:
            city_name=tweet.place.full_name
        except:
            city_name="unknown"

        try:
           country_code=tweet.place.country_code
        except:
            country_code="unknown"    

####        print("entities:",tweet.user.entities)
####        print("url:",tweet.user.url)
####        print(tweet.user.followers_count)
####        print("Account created at: ",tweet.user.created_at)
####        print(tweet.user.friends_count)
####        print(tweet.user.favourites_count)
####        print("geo_enabled",tweet.user.geo_enabled)
####        print(tweet.user.time_zone)
####        print("verified",tweet.user.verified)
####        print(tweet.user.statuses_count)
####        print("lang",tweet.user.lang)
####        print(tweet.user.contributors_enabled)
####        print(tweet.user.is_translator)
####        print(tweet.user.is_translation_enabled)
####        print("following",tweet.user.following)
####        print()


        words_in_description=cleaning(tweet.user.description)
        KEYWORDS_list
##        key__words=['entrepreneur','founder','founded','business owner']
##        if set(key__words) & set(words_in_description.split(' ')):
        
        Istarget=0
        for KEY in KEYWORDS_list:
            if KEY in words_in_description:
                Istarget+=1
        if Istarget>0:
##	  if set(KEYWORDS_list) & set(words_in_description.split(' ')):
            n_target +=1
            print("target detected")
            words_in_tweet=cleaning(tweet.full_text)

            #Open the file to append data
            csvFile_Loc = open(LOC+'.csv', 'a',encoding="utf-8")
            csvWriter = csv.writer(csvFile_Loc, delimiter=';' , quoting=csv.QUOTE_MINIMAL)
            csvWriter.writerow([tweet.user.screen_name.translate(non_bmp_map),
                               tweet.user.name.translate(non_bmp_map),
                               tweet.user.id,
                               tweet.full_text.translate(non_bmp_map),
                               words_in_tweet,
                               tweet.lang,
                               tweet.created_at,
                               city_name,
                               country_code,
                               tweet.user.location.translate(non_bmp_map),
                               tweet.user.description]) 

In [ ]:
print("nember of colleted tweets:",n_total)
print("nember of target tweets:",n_target)
info_csv_Loc = pandas.read_csv(LOC+'.csv', sep=';', na_values=".")
info_csv_Loc.shape 
info_csv_Loc.columns
info_csv_Loc.head()# Preview the first 5 lines of the loaded data 
print(info_csv_Loc['user id'])